<a href="https://colab.research.google.com/github/morningtrading/morningtrading/blob/main/2025test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas
!pip install MetaTrader5

ERROR: Could not find a version that satisfies the requirement MetaTrader5 (from versions: none)
ERROR: No matching distribution found for MetaTrader5


In [5]:
!pip install git+https://github.com/MetaQuotesSoftware/MetaTrader5-Python-API.git # Install from GitHub

  Cloning https://github.com/MetaQuotesSoftware/MetaTrader5-Python-API.git to /tmp/pip-req-build-r5qe3872
  Running command git clone --filter=blob:none --quiet https://github.com/MetaQuotesSoftware/MetaTrader5-Python-API.git /tmp/pip-req-build-r5qe3872
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/MetaQuotesSoftware/MetaTrader5-Python-API.git /tmp/pip-req-build-r5qe3872 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/MetaQuotesSoftware/MetaTrader5-Python-API.git /tmp/pip-req-build-r5qe3872 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

In [3]:
import metaTrader as mt5
import pandas as pd


# function to send a market order
def send_market_order(symbol, volume, order_type, sl=0.0, tp=0.0,
                      deviation=20, comment='', magic=0, type_filling=mt5.ORDER_FILLING_IOC):
    tick = mt5.symbol_info_tick(symbol)

    order_dict = {'buy': 0, 'sell': 1}
    price_dict = {'buy': tick.ask, 'sell': tick.bid}

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_dict[order_type],
        "price": price_dict[order_type],
        "sl": sl,
        "tp": tp,
        "deviation": deviation,
        "magic": magic,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": type_filling,
    }

    order_result = mt5.order_send(request)

    return order_result


def close_position(position, deviation=20, magic=0, comment='', type_filling=mt5.ORDER_FILLING_IOC):
    order_type_dict = {
        0: mt5.ORDER_TYPE_SELL,
        1: mt5.ORDER_TYPE_BUY
    }

    price_dict = {
        0: mt5.symbol_info_tick(position['symbol']).bid,
        1: mt5.symbol_info_tick(position['symbol']).ask
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position['ticket'],  # select the position you want to close
        "symbol": position['symbol'],
        "volume": position['volume'],  # FLOAT
        "type": order_type_dict[position['type']],
        "price": price_dict[position['type']],
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": type_filling,
    }

    print(request)

    order_result = mt5.order_send(request)
    return (order_result)


def close_all_positions(order_type, magic=None, type_filling=mt5.ORDER_FILLING_IOC):
    order_type_dict = {
        'buy': 0,
        'sell': 1
    }

    if mt5.positions_total() > 0:
        positions = mt5.positions_get()

        positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

        # filtering by magic if specified
        if magic:
            positions_df = positions_df[positions_df['magic'] == magic]

        if order_type != 'all':
            positions_df = positions_df[(positions_df['type'] == order_type_dict[order_type])]

        if positions_df.empty:
            print('No open positions')
            return []

        results = []
        for i, position in positions_df.iterrows():
            order_result = close_position(position, type_filling=type_filling)
            print('order_result: ', order_result)
            results.append(order_result)

        return 1


def modify_sl_tp(ticket, stop_loss, take_profit):
    # modify SL/TP

    stop_loss = float(stop_loss)
    take_profit = float(take_profit)


    request = {
        'action': mt5.TRADE_ACTION_SLTP,
        'position': ticket,
        'sl': stop_loss,
        'tp': take_profit
    }

    res = mt5.order_send(request)
    return res


def get_positions(magic=None):
    if mt5.positions_total():
        positions = mt5.positions_get()
        positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

        if magic:
            positions_df = positions_df[positions_df['magic'] == magic]

        return positions_df

    else:
        return pd.DataFrame(columns=['ticket', 'time', 'time_msc', 'time_update', 'time_update_msc', 'type',
                                     'magic', 'identifier', 'reason', 'volume', 'price_open', 'sl', 'tp',
                                     'price_current', 'swap', 'profit', 'symbol', 'comment', 'external_id'])


# Extract Data and Visualization
def get_ohlc_history(symbol, timeframe, date_from, date_to):
    ohlc = mt5.copy_rates_range(symbol, timeframe, date_from, date_to)

    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

def get_tick_history(symbol, start, end):
    ticks = mt5.copy_ticks_range(symbol, start, end, mt5.COPY_TICKS_ALL)
    ticks_df = pd.DataFrame(ticks)
    ticks_df['time'] = pd.to_datetime(ticks_df['time'], unit='s')
    ticks_df = ticks_df[['time', 'bid', 'ask']]
    return ticks_df

ModuleNotFoundError: No module named 'metaTrader'